# Going Back-to-Basics
## Linear methods for regression

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from linear_regression import linear_regression  as lr 
from sklearn import datasets
import pandas as pd

In [2]:
##testing with the prostate dataset
df = pd.read_csv('.//prostate.data',sep='\s+')
#split train and test
df_train = df.loc[df['train'] == 'T']
df_test = df.loc[df['train'] == 'F']
#drop train column
df_train = df_train.drop(['train'],axis=1)
df_test = df_test.drop(['train'],axis=1)
df_train
x_train = df_train[['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']].to_numpy()
y_train = df_train[['lpsa']].to_numpy()
x_test = df_test[['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']].to_numpy()
y_test = df_test[['lpsa']].to_numpy()
predictors = ['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']


In [3]:
#plot correlations between all predictors
#grr = pd.plotting.scatter_matrix(df, figsize=(15, 15), marker='o',
#                                 hist_kwds={'bins': 20}, s=60, alpha=.8)

In [4]:
fh = lr(predictors, x_train, y_train, x_test, y_test, standardize = True, intercept=True)
fh.solve()

In [5]:
fh.backwards_stepwise_selection()

c (30, 9) (9, 1)
c (30, 8) (8, 1)
c (30, 7) (7, 1)
c (30, 6) (6, 1)
c (30, 5) (5, 1)
c (30, 4) (4, 1)
c (30, 3) (3, 1)
c (30, 2) (2, 1)
[29.4263844599084, 29.43730031741708, 30.539778129147354, 32.069447332331784, 32.81499474881555, 34.90774885656786, 37.091845632561345, 44.52858265645385] ['gleason', 'age', 'lcp', 'pgg45', 'lbph', 'svi', 'lweight', 'lcavol']
[16.564820514008186, 16.467503922798873, 17.27080608851507, 15.884637816445682, 14.756207448326178, 13.367761375874872, 16.54138258922808, 15.036170914877058]
